# Mini Project: Deep Learning with Keras

In this mini-project we'll be building a deep learning classifier using Keras to predict income from the popular [Adult Income dataset](http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html).

Predicting income from demographic and socio-economic information is an important task with real-world applications, such as financial planning, market research, and social policy analysis. The Adult dataset, sometimes referred to as the "Census Income" dataset, contains a vast amount of anonymized data on individuals, including features such as age, education, marital status, occupation, and more. Our objective is to leverage this data to train a deep learning model that can effectively predict whether an individual's income exceeds $50,000 annually or not.

Throughout this Colab, we will walk you through the entire process of building a deep learning classifier using Keras, a high-level neural network API that runs on top of TensorFlow. Keras is known for its user-friendly and intuitive interface, making it an excellent choice for both beginners and experienced deep learning practitioners.

Here's a brief outline of what we will cover in this mini-project:

1. **Data Preprocessing:** We will start by loading and exploring the Adult dataset.

2. **Building the Deep Learning Model:** We will construct a neural network using Keras, where we'll dive into understanding the key components of a neural network, including layers, activation functions, and optimization algorithms.

3. **Model Training:** With our model architecture in place, we will split the data into training and validation sets and train the neural network on the training data. We will monitor the training process to prevent overfitting and enhance generalization.

4. **Model Evaluation:** After training, we'll assess the performance of our model on the test dataset.

By the end of this tutorial, you will not only have a functional deep learning classifier for income prediction but also gain valuable insights into how to leverage the power of neural networks for solving real-world classification tasks.


In [393]:
# !pip install scikeras

In [394]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from scikeras.wrappers import KerasClassifier
from sklearn.pipeline import Pipeline

You can download the Adult data from the link [here](https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data).

Here are your tasks:

  1. Load the Adult data into a Pandas Dataframe.
  2. Ensure the dataset has properly named columns. If the columns are not read in, assign them by referencing the dataset documentation.
  3. Display the first five rows of the dataset.

In [395]:
DATA_PATH = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'

# Download the dataset and load it into a pandas DataFrame
# Notes: file does not contain headers, those were added from the website data dictionary
# there are spaces after the commas in the CSV that should be stripped out
col_list = ['age','workclass','fnlweight','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']
df = pd.read_csv(DATA_PATH, skipinitialspace=True, header=None, names=col_list)
df

,age,workclass,fnlweight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [396]:
# Display the first few rows of the DataFrame
df.head()


,age,workclass,fnlweight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


If you're not already familiar with the Adult dataset, it's important to do some exploratory data analysis.

Here are your tasks:

  1. Do exploratory data analysis to give you some better intuition for the dataset. This is a bit open-ended. How many rows/columns are there? How are NULL values represented? What's the percentage of positive cases in the dataset?

  2. Drop all rows with NULL values.

  3. Use Scikit-Learn's [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) to convert the `income` column with a data type string to a binary variable.

In [397]:
# Do some exploratory analysis. How many rows/columns are there? How are NULL
# values represented? What's the percentrage of positive cases in the dataset?
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlweight       32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB
None


In [398]:
print(df.isna().sum())

age               0
workclass         0
fnlweight         0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64


In [399]:
# NULLs are represented by question mark strings: ?
# So count the number of single question marks
# first set up a totals dataframe and zero it out
null_counts = pd.DataFrame(index=range(1), columns=df.columns)
for col in null_counts:
    null_counts[col].values[:] = 0
    
for col in df:
    null_counts[col] = sum([1 for val in df[col] if type(val) == str and val.strip() == "?"])
    
print("Null Counts by column, as indicated by question marks in original data table:")
null_counts.T

Null Counts by column, as indicated by question marks in original data table:


,0
age,0
workclass,1836
fnlweight,0
education,0
education-num,0
marital-status,0
occupation,1843
relationship,0
race,0
sex,0


In [400]:
# but that doesn't tell us how many unique rows have one or more nulls, so do it the mask way
df_null_test = df == "?"
df.mask(df_null_test, inplace=True)
print(f"shape before dropping nulls {df.shape}")
original_shape = df.shape

shape before dropping nulls (32561, 15)


In [401]:
# Find all NULL values and drop them
df.dropna(inplace=True)
clean_shape = df.shape
print(f"shape after dropping nulls {df.shape}")
print(f"Percentage of non-null rows in the dataset is: {clean_shape[0]/original_shape[0]*100:0.0f}%")


shape after dropping nulls (30162, 15)
Percentage of non-null rows in the dataset is: 93%


In [402]:
# Use Scikit-Learn's LabelEncoder to convert the income column with a data type
# string to a binary variable.
le = LabelEncoder()
df['income_enc'] = le.fit_transform(df['income'])
print(df.groupby('income_enc')['age'].count())
print(df.info())

fraction_positive = df['income_enc'].sum()/len(df['income_enc'])
fraction_positive
print(f"Percentage of positive >=50k rows in the dataset is: {fraction_positive*100:0.0f}%")


income_enc
0    22654
1     7508
Name: age, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 30162 entries, 0 to 32560
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             30162 non-null  int64 
 1   workclass       30162 non-null  object
 2   fnlweight       30162 non-null  int64 
 3   education       30162 non-null  object
 4   education-num   30162 non-null  int64 
 5   marital-status  30162 non-null  object
 6   occupation      30162 non-null  object
 7   relationship    30162 non-null  object
 8   race            30162 non-null  object
 9   sex             30162 non-null  object
 10  capital-gain    30162 non-null  int64 
 11  capital-loss    30162 non-null  int64 
 12  hours-per-week  30162 non-null  int64 
 13  native-country  30162 non-null  object
 14  income          30162 non-null  object
 15  income_enc      30162 non-null  int64 
dtypes: int64(7), object(9)
memory usage: 3.9+ 

1. Split the data into training and test sets. Remember not to include the label you're trying to predict, `income`, as a column in your training data.

In [403]:
# Split dataset into training and test sets
# Note: we can do this up front because later on the pipeline will apply the same transforms to both train and test sets equally 
y = df['income_enc'].values
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['income','income_enc']), y, test_size=0.25, random_state=42, stratify=y)

In machine learning, the Receiver Operating Characteristic (ROC) curve and the Area Under the Curve (AUC) metric are commonly used to evaluate the performance of binary classification models. These are valuable tools for understanding how well a model can distinguish between the positive and negative classes in a classification problem.

Let's break down each concept:

1. ROC Curve:
The ROC curve is a graphical representation of a binary classifier's performance as the discrimination threshold is varied. It is created by plotting the True Positive Rate (TPR) against the False Positive Rate (FPR) at different threshold values. Here's how these rates are calculated:

- True Positive Rate (TPR), also called Sensitivity or Recall, measures the proportion of actual positive instances that are correctly identified by the model:
   TPR = True Positives / (True Positives + False Negatives)

- False Positive Rate (FPR) measures the proportion of actual negative instances that are incorrectly classified as positive by the model:
   FPR = False Positives / (False Positives + True Negatives)

The ROC curve is useful because it shows how well a classifier can trade off between sensitivity and specificity across different threshold values. The ideal ROC curve hugs the top-left corner, indicating a high TPR and low FPR, meaning the classifier is excellent at distinguishing between the two classes.

2. AUC (Area Under the Curve):
The AUC is a scalar metric derived from the ROC curve. It represents the area under the ROC curve, hence its name. The AUC ranges from 0 to 1, where 0 indicates a very poor classifier (always predicting the opposite class) and 1 signifies a perfect classifier (making all correct predictions).

The AUC metric is beneficial because it provides a single value to summarize the classifier's overall performance across all possible threshold values. It is particularly useful when dealing with imbalanced datasets, where one class significantly outnumbers the other. In such cases, accuracy alone might not be a reliable evaluation metric, and AUC can provide a more robust performance measure.

A quick rule of thumb for interpreting AUC values:
- AUC ≈ 0.5: The model performs no better than random guessing.
- 0.5 < AUC < 0.7: The model has poor to fair performance.
- 0.7 < AUC < 0.9: The model has good to excellent performance.
- AUC ≈ 1: The model is close to or has a perfect performance.

Here are your tasks:

  1. Use Scikit-Learn's [roc_auc_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html) to calculate the AUC score for a method that always predicts the majority class.  

In [404]:
# Use Scikit-Learn's roc_auc_score to calculate the AUC score for a method that
# always predicts the majority class.

# zero rule algorithm for a classification problem
def predict_majority_class(a_ytrain, a_ytest):
    count_0, count_1 = np.bincount(a_ytrain, minlength=2)
    if count_0 > count_1: 
        majority_class = 0
    else:
        majority_class = 1
    print(f"Majority class for this problem is {majority_class}, of the set {set(a_ytrain)}, with {count_0} zeros and {count_1} ones")
    predicted = np.empty(len(a_ytest))
    predicted.fill(majority_class)
    return predicted
    
baseline_pred = predict_majority_class(y_train, y_test)
baseline_roc_auc = roc_auc_score(y_test, baseline_pred)

print(f"Baseline zero rule algorithm returned shape: {baseline_pred.shape} and ROC/AUC score of {baseline_roc_auc}")

Majority class for this problem is 0, of the set {np.int64(0), np.int64(1)}, with 16990 zeros and 5631 ones
Baseline zero rule algorithm returned shape: (7541,) and ROC/AUC score of 0.5


Now, let's do a little feature engineering.

1. Use Scikit-Learn's [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) to apply One Hot Encoding to the categorical variables in `workclass`, `education`, `marital-status`, `occupation`, `relationship`, 'race', `sex`, and `native-country`. Also, apply [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) to the remaining continuous features. How many columns will the dataframe have after these columns transformations are applied?

In [405]:
# Use Scikit-Learn's ColumnTransformer to apply One Hot Encoding to the
# categorical variables in workclass, education, marital-status, occupation,
# relationship, 'race', sex, and native-country.
# Also, apply MinMaxScaler to
# the remaining continuous features.
categorical_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')

numerical_cols = ['age','fnlweight','education-num','capital-gain','capital-loss','hours-per-week']

num_df = df.drop(columns=categorical_cols+['income', 'income_enc'])

mms = MinMaxScaler().set_output(transform='pandas')

#clean_df = pd.concat([num_df, cat_df], axis=1)
#clean_df
preproc = ColumnTransformer(transformers=[
        ("cats", ohe, categorical_cols),
        ("nums", mms, numerical_cols)],
    verbose=True,
    n_jobs=-1
).set_output(transform='pandas')

preproc.fit_transform(df)


,cats__workclass_Local-gov,cats__workclass_Private,cats__workclass_Self-emp-inc,cats__workclass_Self-emp-not-inc,cats__workclass_State-gov,cats__workclass_Without-pay,cats__education_11th,cats__education_12th,cats__education_1st-4th,cats__education_5th-6th,...,cats__native-country_Trinadad&Tobago,cats__native-country_United-States,cats__native-country_Vietnam,cats__native-country_Yugoslavia,nums__age,nums__fnlweight,nums__education-num,nums__capital-gain,nums__capital-loss,nums__hours-per-week
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.301370,0.043338,0.800000,0.021740,0.0,0.397959
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.452055,0.047277,0.800000,0.000000,0.0,0.122449
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.287671,0.137244,0.533333,0.000000,0.0,0.397959
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.493151,0.150212,0.400000,0.000000,0.0,0.397959
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.150685,0.220703,0.800000,0.000000,0.0,0.397959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.136986,0.165563,0.733333,0.000000,0.0,0.377551
32557,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.315068,0.095589,0.533333,0.000000,0.0,0.397959
32558,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.561644,0.093914,0.533333,0.000000,0.0,0.397959
32559,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.068493,0.127620,0.533333,0.000000,0.0,0.193878


In [406]:
# How many columns will the dataframe have after these columns transformations are applied?
print(f"Encoded and Scaled data frame now has {len(preproc.get_feature_names_out())} columns, excluding the target.")


Encoded and Scaled data frame now has 96 columns, excluding the target.


Keras is an open-source deep learning library written in Python. It was developed to provide a user-friendly, high-level interface for building and training neural networks. The library was created by François Chollet and was first released in March 2015 as part of the Deeplearning4j project. Later, it became part of the TensorFlow ecosystem and is now the official high-level API for TensorFlow.

Keras is designed to be modular, user-friendly, and easy to extend. It allows researchers and developers to quickly prototype and experiment with various deep learning models. One of the primary goals of Keras is to enable fast experimentation, making it simple to build and iterate on different architectures.

Key features of Keras include:

1. User-friendly API: Keras provides a simple and intuitive interface for defining and training deep learning models. Its design philosophy focuses on ease of use and clarity of code.

2. Modularity: Models in Keras are built as a sequence of layers, and users can easily stack, merge, or create complex architectures using a wide range of predefined layers.

3. Extensibility: Keras allows users to define custom layers, loss functions, and metrics. This flexibility enables researchers to experiment with new ideas and algorithms seamlessly.

4. Backends: Initially, Keras supported multiple backends, including TensorFlow, Theano, and CNTK. However, as of TensorFlow version 2.0, TensorFlow has become the primary backend for Keras.

5. Multi-GPU and distributed training: Keras supports training models on multiple GPUs and in distributed computing environments, making it suitable for large-scale experiments.

6. Pre-trained models: Keras includes a collection of pre-trained models for common tasks, such as image classification (e.g., VGG, ResNet, MobileNet) and natural language processing (e.g., Word2Vec, GloVe).

The integration of Keras into TensorFlow as its official high-level API has solidified its position as one of the most popular deep learning libraries in the machine learning community. Its ease of use and versatility have contributed to its widespread adoption in both academia and industry for a wide range of deep learning tasks.

Here are your tasks:

1. Create your own model in Keras to predict income in the Adult training data. Remember, it's always better to start simple and add complexity to the model if necessary. What's a good loss function to use?

2. Keras can be integrated with Scitkit-Learn using a wrapper. Use the [KerasClassifier wrapper](https://adriangb.com/scikeras/stable/generated/scikeras.wrappers.KerasClassifier.html) to integrate your Keras model with the ColumnTransformer from previous steps using a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) object.

3. Fit your model.

4. Calculate the AUC score of your model on the test data. Does the model predict better than random?

5. Generate an ROC curve for your model using [RocCurveDisplay](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.RocCurveDisplay.html). What would the curve look like if all your predictions were randomly generated? What would the curve look like if it you had a perfect model?

In [407]:
# Define the Keras model
# Tried with sigmoid activation and sgd optimizer, but couldn't get an AUC score better than 0.75, this combo is 2 points better.
model = Sequential([
    Input(shape=(96,) ),                # Using input because Python 3.9 tensorflow.keras complains the other ways
    Dense(12, activation='relu'),    # Hidden layer for experimenting; tried 24 nodes, and similar result as 12
    Dense(8, activation='relu'),    # Hidden layer for experimenting; having a 2nd hidden layer doesn't seem to do much to improve accuracy
    Dense(1, activation='relu')      # Output layer needs to be 1 value to match the target dimension
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()



Model: "sequential_35"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_91 (Dense)                │ (None, 12)             │         1,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_92 (Dense)                │ (None, 8)              │           104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_93 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,277 (4.99 KB)

 Trainable params: 1,277 (4.99 KB)

 Non-trainable params: 0 (0.00 B)

In [408]:
# Create a Keras classifier
# note: this is the wrapper class that the keras model goes in to fit into the SciKit Learn Pipeline framework
#   where SciKit Learn classifiers can use the set_params() notation to pass classifier specific arguments, for Keras, they are set in the constructor
clf = KerasClassifier(model=model, epochs=100, verbose=1)



In [409]:
# Create the scikit-learn pipeline
pipe = Pipeline(steps=[
        ("preprocess", preproc),
        ("nnet", clf)],
    verbose=True)


In [ ]:
# Fit the pipeline on the training data
pipe.fit(X_train, y_train)
accuracy = pipe.score(X_test, y_test)
print(f"Final acccuracy: {accuracy}")

[Pipeline] ........ (step 1 of 2) Processing preprocess, total=   1.7s
Epoch 1/100
707/707 ━━━━━━━━━━━━━━━━━━━━ 1s 708us/step - accuracy: 0.7902 - loss: 0.7910
Epoch 2/100
707/707 ━━━━━━━━━━━━━━━━━━━━ 1s 686us/step - accuracy: 0.8276 - loss: 0.3889
Epoch 3/100
707/707 ━━━━━━━━━━━━━━━━━━━━ 1s 715us/step - accuracy: 0.8420 - loss: 0.3583
Epoch 4/100
707/707 ━━━━━━━━━━━━━━━━━━━━ 0s 674us/step - accuracy: 0.8396 - loss: 0.3522
Epoch 5/100
707/707 ━━━━━━━━━━━━━━━━━━━━ 1s 727us/step - accuracy: 0.8405 - loss: 0.3531
Epoch 6/100
707/707 ━━━━━━━━━━━━━━━━━━━━ 1s 687us/step - accuracy: 0.8393 - loss: 0.3454
Epoch 7/100
707/707 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - accuracy: 0.8435 - loss: 0.3434
Epoch 8/100
707/707 ━━━━━━━━━━━━━━━━━━━━ 1s 680us/step - accuracy: 0.8428 - loss: 0.3508
Epoch 9/100
707/707 ━━━━━━━━━━━━━━━━━━━━ 1s 683us/step - accuracy: 0.8472 - loss: 0.3409
Epoch 10/100
707/707 ━━━━━━━━━━━━━━━━━━━━ 1s 686us/step - accuracy: 0.8435 - loss: 0.3487
Epoch 11/100
707/707 ━━━━━━━━━━━━━━━━━

In [ ]:
# Calculate the AUC score of your model on the test data.
# Does the model predict better than random? - Answer: yes, my random test only yielded 50%, so 77% AUC is a better score
y_pred = pipe.predict(X_test)
nn_auc = roc_auc_score(y_test, y_pred)
print(f"Final trained neural network AUC score: {nn_auc}")

In [ ]:
# Generate an ROC curve for your model.

RocCurveDisplay.from_predictions(y_test, y_pred)
